<a href="https://colab.research.google.com/github/Kacper-W-Kozdon/notebook-testing-ivy/blob/main/Sarcasm_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
from google.colab import userdata
import os

In [3]:
files.upload(); #Upload kaggle.json

Saving kaggle.json to kaggle (1).json


In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets list

ref                                                title                                      size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------  ----------------------------------------  -----  -------------------  -------------  ---------  ---------------  
syedanwarafridi/vehicle-sales-data                 Vehicle Sales Data                         19MB  2024-02-21 20:16:17          17247        305  1.0              
sudarshan24byte/online-food-dataset                Online Food Dataset                         3KB  2024-03-02 18:50:30           6909        119  0.9411765        
arnavvvvv/spotify-music                            Top Spotify Songs                          47KB  2024-03-06 05:20:29           5323         73  1.0              
bhavikjikadara/student-study-performance           Student Study Performance                   9KB  2024-03-07 06:14:09           4213         72  1.0              
nbroad/gem

In [6]:
!git clone https://github.com/Kacper-W-Kozdon/demos.git
!git clone https://github.com/Kacper-W-Kozdon/ivy.git
!pip install -U -q ivy accelerate>=0.21.0 mlflow datasets>=2.14.5 nlp 2>/dev/null

Cloning into 'demos'...
remote: Enumerating objects: 3152, done.
remote: Counting objects: 100% (608/608), done.
remote: Compressing objects: 100% (219/219), done.
remote: Total 3152 (delta 447), reused 438 (delta 358), pack-reused 2544
Receiving objects: 100% (3152/3152), 16.15 MiB | 11.27 MiB/s, done.
Resolving deltas: 100% (1820/1820), done.
Cloning into 'ivy'...
remote: Enumerating objects: 188936, done.
remote: Counting objects: 100% (733/733), done.
remote: Compressing objects: 100% (324/324), done.
remote: Total 188936 (delta 505), reused 556 (delta 409), pack-reused 188203
Receiving objects: 100% (188936/188936), 164.28 MiB | 8.84 MiB/s, done.
Resolving deltas: 100% (152149/152149), done.


In [4]:
import ivy

In [8]:
!kaggle datasets download -d danofer/sarcasm
!cp -f sarcasm.zip '/content/demos/Contributor_demos/Sarcasm Detection/'
!unzip '/content/demos/Contributor_demos/Sarcasm Detection/sarcasm.zip' -d '/content/demos/Contributor_demos/Sarcasm Detection/'

 97% 210M/216M [00:03<00:00, 121MB/s] 
100% 216M/216M [00:03<00:00, 72.7MB/s]
Archive:  /content/demos/Contributor_demos/Sarcasm Detection/sarcasm.zip
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/test-balanced.csv  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/test-unbalanced.csv  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/train-balanced-sarc.csv.gz  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/train-balanced-sarcasm.csv  


In [5]:
# Import necessary libraries
import pandas as pd  # For data manipulation and analysis
import gc  # For garbage collection to manage memory
import re  # For regular expressions
import numpy as np  # For numerical operations and arrays

import warnings  # For handling warnings
warnings.filterwarnings("ignore")  # Ignore warning messages

import torch  # PyTorch library for deep learning
from transformers import AutoModel, AutoTokenizer  # Transformers library for natural language processing
from transformers import TextDataset, LineByLineTextDataset, DataCollatorForLanguageModeling, \
pipeline, Trainer, TrainingArguments, DataCollatorWithPadding  # Transformers components for text processing
from transformers import AutoModelForSequenceClassification  # Transformer model for sequence classification

import accelerate

from nlp import Dataset  # Import custom 'Dataset' class for natural language processing tasks
from imblearn.over_sampling import RandomOverSampler  # For oversampling to handle class imbalance
import datasets  # Import datasets library
from datasets import Dataset, Image, ClassLabel  # Import custom 'Dataset', 'ClassLabel', and 'Image' classes
from transformers import pipeline  # Transformers library for pipelines
from bs4 import BeautifulSoup  # For parsing HTML content

import matplotlib.pyplot as plt  # For data visualization
import itertools  # For working with iterators
from sklearn.metrics import (  # Import various metrics from scikit-learn
    accuracy_score,  # For calculating accuracy
    roc_auc_score,  # For ROC AUC score
    confusion_matrix,  # For confusion matrix
    classification_report,  # For classification report
    f1_score  # For F1 score
)

from datasets import load_metric  # Import load_metric function to load evaluation metrics

from tqdm import tqdm  # For displaying progress bars
tqdm.pandas()  # Enable progress bars for pandas operations

In [85]:
df = pd.read_csv("/content/demos/Contributor_demos/Sarcasm Detection/train-balanced-sarcasm.csv")
df = df.drop_duplicates()

In [114]:
df = df[["comment", 'label']]
df = df.rename(columns={"comment": "title"})
df = df[~df["title"].isnull()]
df = df[~df['label'].isnull()]
print(df.sample(15))

                                                    title  label
61955                But did he use the proper buzzwords?      1
206289  It's not like any of the schools could benefit...      1
176033                 And make him grow +10/+10 per turn      1
762832  I walked by a construction site on my way to g...      1
350503  Side this is your first time, for next time I'...      0
630138     nope..this is a completely natural phenomenon.      1
687857  James, don't tell Audrey it's "solid" evidence...      0
637111                                                idc      0
37446                          right, cant have 2 Stuarts      0
615956     I don't know who is down voting you, but this!      1
853348         Wow OP...get ready to be downvoted to hell      1
717524      It is extremely worth the wait, I assure you.      0
617726                                      Good to hear.      0
143048  We didn't get a great look at the field, did n...      0
198980  I just bent it ba

In [115]:
dataset = Dataset.from_pandas(df[["title"]].sample(100))
print(f"Dataset info: {dataset.info}", "\n", f"Dataset size: {dataset.__len__()}")


Dataset info: DatasetInfo(description='', citation='', homepage='', license='', features={'title': Value(dtype='string', id=None), '__index_level_0__': Value(dtype='int64', id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name=None, dataset_name=None, config_name=None, version=None, splits=None, download_checksums=None, download_size=None, post_processing_size=None, dataset_size=None, size_in_bytes=None) 
 Dataset size: 100


In [9]:
print(dataset["title"][:5])

['NC and NH.', 'You do know west teams play against west teams more than east teams right?', "They were underdogs earlier today, but since Gronk's announcement this afternoon, the Vegas line has moved to patriots -1", 'This meme isn\'t funny none of the "new york nigga" ones are.', 'I could use one of those tools.']


In [10]:
del df
gc.collect()

5

In [117]:
def get_training_corpus():
  training_corpus = (
    dataset["title"][i : i + 1000]
    for i in range(0, len(dataset["title"]), 1000)
  )
  return training_corpus

training_corpus = get_training_corpus()


In [ ]:
#Training of the tokenizer.
old_tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)
tokenizer.save_pretrained("Sarcasm_Detection-Tokenizer")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [12]:
files.upload(); #Upload Sarcasm_Detection-Tokenizer.zip if already trained -> '/content/Sarcasm Detection/'

Saving Sarcasm_Detection-Tokenizer.zip to Sarcasm_Detection-Tokenizer (1).zip


In [17]:
!unzip '/content/Sarcasm_Detection-Tokenizer.zip' -d '/content/demos/Contributor_demos/Sarcasm Detection/'

Archive:  /content/Sarcasm_Detection-Tokenizer.zip
   creating: /content/demos/Contributor_demos/Sarcasm Detection/Sarcasm_Detection-Tokenizer/
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/Sarcasm_Detection-Tokenizer/kaggle.json  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/Sarcasm_Detection-Tokenizer/merges.txt  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/Sarcasm_Detection-Tokenizer/special_tokens_map.json  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/Sarcasm_Detection-Tokenizer/tokenizer.json  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/Sarcasm_Detection-Tokenizer/tokenizer_config.json  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/Sarcasm_Detection-Tokenizer/vocab.json  


In [88]:
tokenizer = AutoTokenizer.from_pretrained("/content/demos/Contributor_demos/Sarcasm Detection/Sarcasm_Detection-Tokenizer")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
if tokenizer.sep_token is None:
    tokenizer.add_special_tokens({'sep_token': '[SEP]'})
print(dataset["title"][1], dataset["label"][1])
tokens = tokenizer(dataset["title"][1:5], padding=True)
print(tokens)
decoded_string = tokenizer.decode(tokens.get("input_ids")[0])
print(decoded_string)

lil babby hands 0
{'input_ids': [[49641, 30934, 2413, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000], [6850, 284, 6304, 12, 1157, 304, 3136, 14, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000], [3722, 17790, 44, 10868, 9482, 29928, 10868, 9482, 29928, 10868, 9482, 29928, 10868, 9482, 29928, 10868, 9482, 29928, 10868, 9482, 29928, 10868, 9482, 29928, 10868, 9482, 29928, 10868, 9482, 29928, 10868], [5561, 464, 14, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000]], 'attention_mask': [[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 

In [118]:
def tokenize_function(examples):
    return tokenizer(examples["title"], padding = "max_length", truncation=True, return_tensors="pt")

def get_tokenized_dataset():
  tokenized_dataset = (
    tokenize_function(dataset["title"][i : i + 1000])
    for i in range(0, len(dataset["title"]), 1000)
  )
  return tokenized_dataset


tokenized_dataset = get_tokenized_dataset()

training_corpus = get_training_corpus()

In [ ]:
def get_labels():
  labels = (
    dataset["label"][i : i + 1000]
    for i in range(0, len(dataset["label"]), 1000)
  )
  return labels
labels = get_labels()

In [71]:
batch = next(tokenized_dataset)
print(batch[0].ids)
for token in batch[:]:
  print(token.ids)
  break
tr_batch = {"token_ids": [token.ids for token in batch[:]], "attention_mask": [token.attention_mask for token in batch[:]]}
print(tr_batch["token_ids"][:2])
tr_dset = Dataset.from_dict(tr_batch)
print(tr_dset)

TypeError: PreTrainedTokenizerFast._batch_encode_plus() got an unexpected keyword argument 'batched'

In [16]:
model = AutoModel.from_pretrained("gpt2")

In [33]:
print(model.config)
print(dir(model))
print(model.main_input_name)
print(tokenizer.model_input_names)
print(dir(tokenizer))

GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.38.2",
  "use_cache": true,
  "vocab_size": 50257
}

['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__'

In [22]:
!accelerate config default

accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [208]:
tokenized_dataset = get_tokenized_dataset()
labels = get_labels()

In [17]:
dataset_len = dataset.__len__()
num_train_epochs = 3
max_steps = num_train_epochs * dataset_len
training_args = TrainingArguments(output_dir="/content/demos/Contributor_demos/Sarcasm Detection/", ) #Requires enabling GPU

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy_score(labels, predictions)

In [112]:
print(tokenizer(dataset[:3]["title"]))
example = next(tokenized_dataset)
print(example[:3])

{'input_ids': [[5618, 307, 29473, 14], [446, 363, 465, 5140, 2347, 528, 1095, 5140, 2347, 506, 525, 5338, 2347, 539, 31], [741, 631, 42310, 4887, 1953, 12, 392, 1014, 18461, 312, 10837, 371, 15283, 12, 265, 9009, 1698, 515, 5277, 284, 14656, 742, 17]], 'attention_mask': [[1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
[Encoding(num_tokens=1024, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]), Encoding(num_tokens=1024, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]), Encoding(num_tokens=1024, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]


In [119]:
training_dataset = dataset
print(type(dataset))
training_dataset = training_dataset.map(tokenize_function, batched=True)
print(training_dataset)

<class 'datasets.arrow_dataset.Dataset'>


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['title', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 100
})


In [120]:
model(**training_dataset[0])

TypeError: GPT2Model.forward() got an unexpected keyword argument 'title'

In [105]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training_dataset,
    compute_metrics=compute_metrics,
  )
trainer.train()

IndexError: index out of range in self

In [58]:
tokenizer.save_pretrained("Sarcasm_Detection-Tokenizer")
model.save_pretrained("Sarcasm_Detection-Model")

TypeError: object of type 'generator' has no len()

In [23]:
from ivy.stateful.module import Module
from ivy.stateful.sequential import Sequential
from ivy.stateful.layers import *
from ivy.stateful.losses import *
from ivy.stateful.optimizers import *
from ivy.stateful.activations import *
from ivy.stateful.initializers import *
from ivy.stateful.norms import *

# IVY TRANSPILER

Installing kaggle and uploading the API key necessary to use it.

In [ ]:
!pip install -q kaggle
from google.colab import files
from google.colab import userdata
import os
files.upload(); #Upload kaggle.json - you can get from the kaggle account settings, from the API section.

To use the API, credentials need to be copied into the kaggle folder. If everything works, the output will show the list of available datasets.

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

Preparing the ivy library.

In [123]:
#Inser the correct user when cloning the repos. Make sure that they are up-to-date.

!git clone "https://github.com/Kacper-W-Kozdon/demos.git"
!git clone "https://github.com/Kacper-W-Kozdon/ivy.git"
!pip install -U -q ivy accelerate>=0.21.0 mlflow datasets>=2.14.5 nlp 2>/dev/null

fatal: destination path 'demos' already exists and is not an empty directory.
fatal: destination path 'ivy' already exists and is not an empty directory.


Next: import the ivy library and get the dataset.

In [ ]:
import ivy
!kaggle datasets download -d danofer/sarcasm
!cp -f sarcasm.zip '/content/demos/Contributor_demos/Sarcasm Detection/'
!unzip '/content/demos/Contributor_demos/Sarcasm Detection/sarcasm.zip' -d '/content/demos/Contributor_demos/Sarcasm Detection/'

Import the libraries suggested in the model which is to be transpiled.

In [ ]:
# Import necessary libraries
import pandas as pd  # For data manipulation and analysis
import gc  # For garbage collection to manage memory
import re  # For regular expressions
import numpy as np  # For numerical operations and arrays

import warnings  # For handling warnings
warnings.filterwarnings("ignore")  # Ignore warning messages

import torch  # PyTorch library for deep learning
from transformers import AutoModel, AutoTokenizer  # Transformers library for natural language processing
from transformers import TextDataset, LineByLineTextDataset, DataCollatorForLanguageModeling, \
pipeline, Trainer, TrainingArguments, DataCollatorWithPadding  # Transformers components for text processing
from transformers import AutoModelForSequenceClassification  # Transformer model for sequence classification

import accelerate

from nlp import Dataset  # Import custom 'Dataset' class for natural language processing tasks
from imblearn.over_sampling import RandomOverSampler  # For oversampling to handle class imbalance
import datasets  # Import datasets library
from datasets import Dataset, Image, ClassLabel  # Import custom 'Dataset', 'ClassLabel', and 'Image' classes
from transformers import pipeline  # Transformers library for pipelines
from bs4 import BeautifulSoup  # For parsing HTML content

import matplotlib.pyplot as plt  # For data visualization
import itertools  # For working with iterators
from sklearn.metrics import (  # Import various metrics from scikit-learn
    accuracy_score,  # For calculating accuracy
    roc_auc_score,  # For ROC AUC score
    confusion_matrix,  # For confusion matrix
    classification_report,  # For classification report
    f1_score  # For F1 score
)

from datasets import load_metric  # Import load_metric function to load evaluation metrics

from tqdm import tqdm  # For displaying progress bars
tqdm.pandas()  # Enable progress bars for pandas operations

Get the API key for ivy transpiler from your account and upload it to the project. Move it to the correct directory.

In [ ]:
files.upload(); #Upload key.pem - you can get from the kaggle account settings, from the API section.

In [124]:
!mkdir ~/.ivy #It might be necessary to change ".ivy" to "ivy".
!cp key.pem ~/.ivy/

cp: cannot stat 'key.pem': No such file or directory
